In [11]:
import os
from ultralytics import YOLO

path = '../images/cropped_scales/random'
images = [os.path.join(path, img) for img in os.listdir(path) if not img.endswith('only.jpg')]

model = YOLO('../models/classify_scale_medium_v2.pt')

results = model(images, save=True, save_crop=True, conf=0.5, retina_masks=True)


0: 640x640 1 scale_random, 6.1ms
1: 640x640 1 scale_random, 6.1ms
2: 640x640 1 scale_random, 6.1ms
3: 640x640 1 scale_random, 6.1ms
4: 640x640 1 scale_random, 6.1ms
5: 640x640 1 scale_random, 6.1ms
6: 640x640 1 scale_random, 6.1ms
7: 640x640 1 scale_random, 6.1ms
8: 640x640 1 scale_random, 6.1ms
9: 640x640 1 scale_random, 6.1ms
10: 640x640 1 scale_random, 6.1ms
11: 640x640 1 scale_random, 6.1ms
12: 640x640 1 scale_random, 6.1ms
13: 640x640 1 scale_random, 6.1ms
14: 640x640 2 scale_randoms, 6.1ms
15: 640x640 1 scale_random, 6.1ms
16: 640x640 1 scale_random, 6.1ms
17: 640x640 1 scale_random, 6.1ms
18: 640x640 1 scale_random, 6.1ms
19: 640x640 1 scale_random, 6.1ms
20: 640x640 1 scale_random, 6.1ms
21: 640x640 1 scale_random, 6.1ms
22: 640x640 1 scale_random, 6.1ms
23: 640x640 1 scale_random, 6.1ms
24: 640x640 1 scale_random, 6.1ms
25: 640x640 1 scale_random, 6.1ms
26: 640x640 1 scale_random, 6.1ms
27: 640x640 1 scale_fixed, 1 scale_random, 6.1ms
28: 640x640 1 scale_random, 6.1ms
29: 640